In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Amyotrophic_Lateral_Sclerosis/GSE139384'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Synaptopathy in Kii ALS/PDC, a disease concept based on transcriptome analyses of human brains"
!Series_summary	"Amyotrophic lateral sclerosis (ALS) and parkinsonism-dementia complex (PDC) (ALS/PDC) is a unique endemic neurodegenerative disease, with high-incidence foci in the Kii Peninsula, Japan. Although ALS/PDC presents with multiple proteinopathies, the genetic and environmental factors that influence disease onset remain unknown. We performed transcriptome analyses of patients’ brains, which may provide new insights into the pathomechanisms underlying Kii ALS/PDC."
!Series_summary	"We prepared frozen brains from 3 healthy controls (frontal lobe and temporal lobe), 3 patients with Alzheimer’s disease (AD) (frontal lobe and temporal lobe) as tauopathy-disease controls, and 21 patients with Kii ALS/PDC (frontal lobe and/or temporal lobe). We acquired microarray data from the cerebral gray and white matter tissues of Kii ALS/PDC patients."
!Seri

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Based on the RNA extraction and microarray data mention

# Identify keys for each variable in the sample characteristics dictionary
trait_row = None  # No explicit marker for ALS in the sample characteristics dictionary
age_row = 2  # Age values are explicitly available
gender_row = 2  # Gender values are explicitly available

# Define functions for data type conversion
def convert_trait(value):
    return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender_str = value.split(':')[1].strip()
        return 1 if gender_str.lower() == 'male' else 0 if gender_str.lower() == 'female' else None
    except:
        return None

# Save cohort information
save_cohort_info('GSE139384', './preprocessed/Amyotrophic_Lateral_Sclerosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Amyotrophic_Lateral_Sclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Amyotrophic_Lateral_Sclerosis/trait_data/GSE139384.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Amyotrophic_Lateral_Sclerosis/cohort_info.json
